# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     - ----------------------------------- 30.7/990.0 kB 640.0 kB/s eta 0:00:02
     -- ---------------------------------- 61.4/990.0 kB 812.7 kB/s eta 0:00:02
     ---- ------------------------------- 122.9/990.0 kB 901.1 kB/s eta 0:00:01
     ---------- --------------------------- 276.5/990.0 kB 1.5 MB/s eta 0:00:01
     ---------------- --------------------- 440.3/990.0 kB 2.1 MB/s eta 0:00:01
     ---------------------------------- --- 911.4/990.0 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 990.0/990.0 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/373.5 kB ? eta -:--:--
     ------------------------------------- 373.5/373.5 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.20
    Uninstalling langchain-core-0.2.20:
      Successfully 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [4]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [5]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很严谨的生物学家。
你擅长用简单易懂的方式来幽默地阐述关于生物学的复杂话题。
在面对你不熟悉的生物学问题时，你会坦诚地承认。

这是一个问题：
{input}"""

computer_template = """你是一位很优秀的计算机教授。
你热衷于探索计算机科学的奥秘，并且用计算机思维和逻辑回答问题，
你回答问题时还常常探讨如何利用计算机技术改善人们的日常生活和工作。

这是一个问题：
{input}"""

chinese_template = """你是一位很博学的汉语文学老师。
你善于清晰地讲解复杂的文学概念和作品。
如果碰到超出你知识范围的问题，你会毫不犹豫地承认。

这是一个问题：
{input}"""

In [6]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
        {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
        {
        "name": "汉语文学",
        "description": "适用于回答汉语文学问题",
        "prompt_template": chinese_template,
    },
]

In [32]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1", api_key="sk-YIOnMFnuWMDaLG0bC93a20F2456b4dAdAa4aE692449483E7")

In [33]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [34]:
destination_chains

{'物理': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位非常聪明的物理教授。\n你擅长以简洁易懂的方式回答关于物理的问题。\n当你不知道某个问题的答案时，你会坦诚承认。\n\n这是一个问题：\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001D1557AAC80>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001D1562F56F0>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.xiaoai.plus/v1', openai_proxy='')),
 '数学': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位很棒的数学家。你擅长回答数学问题。\n之所以如此出色，是因为你能够将难题分解成各个组成部分，\n先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。\n\n这是一个问题：\n{input}'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001D1557AAC80>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001D1562F56F0>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.xiaoai.plus/v1', openai_proxy='')),
 '生物': LLMChain(prompt=PromptTemplate(input_variables=['input'], template='你是一位很严谨的生物学家

In [35]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [36]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [37]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [38]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语文学: 适用于回答汉语文学问题']


In [39]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文学: 适用于回答汉语文学问题


In [40]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [41]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语文学: 适用于回答汉语文学问题

<< INPUT >>
{input}

<< 

In [42]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [43]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射指的是一种理想的热辐射体，它能够以任何温度下都会发出电磁波，其强度和频率分布与温度有关。它是一种重要的概念，能够帮助我们理解热力学和量子力学。'}


In [44]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '41'}
> Finished chain.
{'input': '41', 'text': ' + 231\n\n41 + 231 = 272\n\n首先，我们可以将41和231分别拆分为40+1和200+30+1，这样我们就可以先回答40+200=240和1+30=31，\n然后将它们相加得到最终的答案272。这种分解问题的方法可以帮助我们更轻松地解决复杂的数学问题。希望你能继续保持对数学的热爱，不断挑战自己，成为更优秀的数学家！'}


In [45]:
router_chain.verbose = True

In [46]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么?'}
> Finished chain.
{'input': '黑洞是什么?', 'text': '\n\n黑洞是一种极其密集的天体，它的引力非常强大，甚至连光都无法逃逸。它的形成是由于恒星在死亡过程中，质量过大导致它的引力无法抵抗，最终导致它坍缩成一个无穷小的点，即为黑洞。黑洞是宇宙中最神秘的物体之一，我们还有很多不了解的地方。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [47]:
print(chain.invoke("请介绍一下食物链的金字塔"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '请介绍一下食物链的金字塔'}
> Finished chain.
{'input': '请介绍一下食物链的金字塔', 'text': '结构。\n\n食物链的金字塔结构是一种生态学概念，用来描述生物之间的食物关系。它通常是从底部开始，由植物开始，然后是食草动物，食肉动物，以及最终的食肉动物。这种结构反映了食物在生态系统中的传递和转化过程。简单来说，就是越往上层，食物链中的动物越大，体量越大，食量也越大。所以，金字塔的顶端往往是顶级捕食者，如狮子、老虎等。而底部则是植物，它们通过光合作用将太阳能转化为营养物质，为整个食物链提供能量。这个金字塔结构还有一个重要的概'}


In [48]:
print(chain.invoke("计算机网络的七层协议分别是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机网络的七层协议分别是什么？'}
> Finished chain.
{'input': '计算机网络的七层协议分别是什么？', 'text': '\n\n答案：\n1. 物理层\n2. 数据链路层\n3. 网络层\n4. 传输层\n5. 会话层\n6. 表示层\n7. 应用层'}


In [50]:
print(chain.invoke("近现代有哪些作品值得阅读？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语文学: {'input': '近现代有哪些作品值得阅读？'}
> Finished chain.
{'input': '近现代有哪些作品值得阅读？', 'text': '\n\n这是一个有待讨论和探索的问题，因为近现代的作品数量众多，每个人的阅读偏好也不同，无法给出一个固定的答案。但是可以提供一些经典的作品推荐，如《围城》、《活着》、《白鹿原》等。另外，根据自己的兴趣和需求，也可以寻找一些专门的文学推荐书籍或者咨询图书馆、书店的工作人员。'}
